# 方差检验

In [4]:
from scipy.stats import f_oneway

# 三组样本数据
data_group1 = [-0.98497713, -0.98406264, -0.97766128, -0.96028704, -0.92645669,
       -0.87069302, -0.78752964, -0.67151735, -0.51723268, -0.31928901,
       -0.07235114,  0.22884586,  0.58947204,  1.01457096,  1.50901915,
        2.07747472,  2.72431311,  3.45354765,  4.26873316,  5.17285063,
        6.16817226,  7.25610683,  8.43702702,  9.71008265, 11.07300618,
       12.5219203 , 14.05116077, 15.65313147, 17.31821131, 19.03473459,
       20.7890662 , 22.56578972, 24.34802027, 26.11784366, 27.8568707 ,
       29.54688086, 31.17051532, 32.71196882, 34.15762395, 35.49657342,
       36.72098532]
data_group2 = [-0.98497713, -0.98392233, -0.97653883, -0.95649908, -0.91747921,
       -0.85316342, -0.75725035, -0.62346172, -0.44555404, -0.21733394,
        0.06732147,  0.41443782,  0.82990841,  1.31944849,  1.88853644,
        2.5423389 ,  3.28561681,  4.12260964,  5.05689511,  6.09122289,
        7.22732204,  8.46568422,  9.80532754, 11.2435502 , 12.77568716,
       14.39488888, 16.09194584, 17.85518693, 19.67048189, 21.52137678,
       23.38938553, 25.25444961, 27.09556143, 28.89152701, 30.62182208,
       32.26747744, 33.81191722, 35.24167246, 36.54690198, 37.72167329,
       38.76398449]
data_group3 = [-9.84977131e-01, -9.83572938e-01, -9.73743819e-01, -9.47066848e-01,
       -8.95125675e-01, -8.09518337e-01, -6.81868767e-01, -5.03842841e-01,
       -2.67170326e-01,  3.63252981e-02,  4.14686430e-01,  8.75766865e-01,
        1.42715896e+00,  2.07609743e+00,  2.82933419e+00,  3.69297845e+00,
        4.67229653e+00,  5.77146714e+00,  6.99329007e+00,  8.33885014e+00,
        9.80714412e+00,  1.13946859e+01,  1.30951148e+01,  1.48988441e+01,
        1.67927947e+01,  1.87602717e+01,  2.07810402e+01,  2.28316550e+01,
        2.48860788e+01,  2.69165965e+01,  2.88949907e+01,  3.07939001e+01,
        3.25882406e+01,  3.42565432e+01,  3.57820579e+01,  3.71534980e+01,
        3.83653460e+01,  3.94177048e+01,  4.03157398e+01,  4.10688073e+01,
        4.16893947e+01]

# 执行方差分析
f_statistic, p_value = f_oneway(data_group1, data_group2, data_group3)

# 输出结果
print("F-statistic:", f_statistic)
print("P-value:", p_value)

# 判断是否拒绝零假设
alpha = 0.05
if p_value < alpha:
    print("拒绝零假设，表明至少有一个总体的方差不相等。")
else:
    print("接受零假设，表明各总体的方差相等。")

F-statistic: 0.7886660384329468
P-value: 0.45679172686326475
接受零假设，表明各总体的方差相等。


In [5]:
from scipy.stats import ttest_ind

# 两组样本数据
data_group1 = [-0.98497713, -0.98406264, -0.97766128, -0.96028704, -0.92645669,
       -0.87069302, -0.78752964, -0.67151735, -0.51723268, -0.31928901,
       -0.07235114,  0.22884586,  0.58947204,  1.01457096,  1.50901915,
        2.07747472,  2.72431311,  3.45354765,  4.26873316,  5.17285063,
        6.16817226,  7.25610683,  8.43702702,  9.71008265, 11.07300618,
       12.5219203 , 14.05116077, 15.65313147, 17.31821131, 19.03473459,
       20.7890662 , 22.56578972, 24.34802027, 26.11784366, 27.8568707 ,
       29.54688086, 31.17051532, 32.71196882, 34.15762395, 35.49657342,
       36.72098532]
data_group2 = [-0.98497713, -0.98392233, -0.97653883, -0.95649908, -0.91747921,
       -0.85316342, -0.75725035, -0.62346172, -0.44555404, -0.21733394,
        0.06732147,  0.41443782,  0.82990841,  1.31944849,  1.88853644,
        2.5423389 ,  3.28561681,  4.12260964,  5.05689511,  6.09122289,
        7.22732204,  8.46568422,  9.80532754, 11.2435502 , 12.77568716,
       14.39488888, 16.09194584, 17.85518693, 19.67048189, 21.52137678,
       23.38938553, 25.25444961, 27.09556143, 28.89152701, 30.62182208,
       32.26747744, 33.81191722, 35.24167246, 36.54690198, 37.72167329,
       38.76398449]
data_group3 = [-9.84977131e-01, -9.83572938e-01, -9.73743819e-01, -9.47066848e-01,
       -8.95125675e-01, -8.09518337e-01, -6.81868767e-01, -5.03842841e-01,
       -2.67170326e-01,  3.63252981e-02,  4.14686430e-01,  8.75766865e-01,
        1.42715896e+00,  2.07609743e+00,  2.82933419e+00,  3.69297845e+00,
        4.67229653e+00,  5.77146714e+00,  6.99329007e+00,  8.33885014e+00,
        9.80714412e+00,  1.13946859e+01,  1.30951148e+01,  1.48988441e+01,
        1.67927947e+01,  1.87602717e+01,  2.07810402e+01,  2.28316550e+01,
        2.48860788e+01,  2.69165965e+01,  2.88949907e+01,  3.07939001e+01,
        3.25882406e+01,  3.42565432e+01,  3.57820579e+01,  3.71534980e+01,
        3.83653460e+01,  3.94177048e+01,  4.03157398e+01,  4.10688073e+01,
        4.16893947e+01]

# 执行 t 检验
t_statistic, p_value = ttest_ind(data_group1, data_group2)

# 输出结果
print("T-statistic:", t_statistic)
print("P-value:", p_value)

# 判断是否拒绝零假设
alpha = 0.05
if p_value < alpha:
    print("拒绝零假设，表明两组样本均值存在显著差异。")
else:
    print("接受零假设，表明两组样本均值相等。")

T-statistic: -0.4292333540305476
P-value: 0.668907172701248
接受零假设，表明两组样本均值相等。


In [16]:
from scipy.stats import wilcoxon, ttest_rel

# 两组相关样本数据
data_group1 = [-0.98497713, -0.98406264, -0.97766128, -0.96028704, -0.92645669,
       -0.87069302, -0.78752964, -0.67151735, -0.51723268, -0.31928901,
       -0.07235114,  0.22884586,  0.58947204,  1.01457096,  1.50901915,
        2.07747472,  2.72431311,  3.45354765,  4.26873316,  5.17285063,
        6.16817226,  7.25610683,  8.43702702,  9.71008265, 11.07300618,
       12.5219203 , 14.05116077, 15.65313147, 17.31821131, 19.03473459,
       20.7890662 , 22.56578972, 24.34802027, 26.11784366, 27.8568707 ,
       29.54688086, 31.17051532, 32.71196882, 34.15762395, 35.49657342,
       36.72098532]
data_group2 = [-0.98497713, -0.98392233, -0.97653883, -0.95649908, -0.91747921,
       -0.85316342, -0.75725035, -0.62346172, -0.44555404, -0.21733394,
        0.06732147,  0.41443782,  0.82990841,  1.31944849,  1.88853644,
        2.5423389 ,  3.28561681,  4.12260964,  5.05689511,  6.09122289,
        7.22732204,  8.46568422,  9.80532754, 11.2435502 , 12.77568716,
       14.39488888, 16.09194584, 17.85518693, 19.67048189, 21.52137678,
       23.38938553, 25.25444961, 27.09556143, 28.89152701, 30.62182208,
       32.26747744, 33.81191722, 35.24167246, 36.54690198, 37.72167329,
       38.76398449]
data_group3 = [-9.84977131e-01, -9.83572938e-01, -9.73743819e-01, -9.47066848e-01,
       -8.95125675e-01, -8.09518337e-01, -6.81868767e-01, -5.03842841e-01,
       -2.67170326e-01,  3.63252981e-02,  4.14686430e-01,  8.75766865e-01,
        1.42715896e+00,  2.07609743e+00,  2.82933419e+00,  3.69297845e+00,
        4.67229653e+00,  5.77146714e+00,  6.99329007e+00,  8.33885014e+00,
        9.80714412e+00,  1.13946859e+01,  1.30951148e+01,  1.48988441e+01,
        1.67927947e+01,  1.87602717e+01,  2.07810402e+01,  2.28316550e+01,
        2.48860788e+01,  2.69165965e+01,  2.88949907e+01,  3.07939001e+01,
        3.25882406e+01,  3.42565432e+01,  3.57820579e+01,  3.71534980e+01,
        3.83653460e+01,  3.94177048e+01,  4.03157398e+01,  4.10688073e+01,
        4.16893947e+01]

# 执行 Wilcoxon 秩和检验
statistic, p_value = wilcoxon(data_group3, data_group2)
statistic, p_value = ttest_rel(data_group1, data_group3)

# 输出结果
print("Wilcoxon statistic:", statistic)
print("P-value:", p_value)

# 判断是否拒绝零假设
alpha = 0.05
if p_value < alpha:
    print("拒绝零假设，表明两组相关样本之间存在显著差异。")
else:
    print("接受零假设，表明两组相关样本之间没有显著差异。")

Wilcoxon statistic: -7.708753260452694
P-value: 1.970353709563622e-09
拒绝零假设，表明两组相关样本之间存在显著差异。


In [ ]:
from scipy.stats import chi2_contingency

# 创建一个包含分类数据的二维表格（列联表）
# 例如，这里是一个 2x2 的表格
observed = [data_group1,
            data_group2]

# 执行卡方检验
chi2_stat, p_val, dof, expected = chi2_contingency(observed)

# 输出结果
print("Chi-square statistic:", chi2_stat)
print("P-value:", p_val)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

# 判断是否拒绝零假设
alpha = 0.05
if p_val < alpha:
    print("拒绝零假设，表明两组分类数据之间存在显著差异。")
else:
    print("接受零假设，表明两组分类数据之间没有显著差异。")

# 数据对比debug

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [39]:
old_file = Path("./cache/Chinese_extract_experiment_df_average_freq.csv")
new_file = Path("./cache/Chinese_extract_experiment_df_average_freq_1234w.csv")
old_df = pd.read_csv(old_file, header=0)
new_df = pd.read_csv(new_file, header=0)

In [40]:
(old_df.NIHL1234 == new_df.NIHL1234).value_counts()

True    4602
Name: NIHL1234, dtype: int64

In [41]:
(new_df.NIHL1234_Y == new_df.NIHL1234w_Y).value_counts()

True     4513
False      89
dtype: int64

In [42]:
(new_df.NIHL1234_Y == old_df.NIHL1234_Y).value_counts()

True    4602
Name: NIHL1234_Y, dtype: int64